### Guide to that notebook: 
1. Download cryptocurrency data from https://www.CryptoDataDownload.com. Combine all files into one dataset and remove any unnecessary columns.  
2. Create buttons for different time ranges and a dropdown with various cryptocurrency names for the future charts. 
3. Create two charts demonstrating  trends: candlestick and barchart. Customise them.
4. Launch the dashboard. 

#### Import necessary libraries. 

In [1]:
import pandas as pd 
# The pandas datareader library allows the user to extract financial data from various internet sources 
import pandas_datareader.data as web
# datetime module provides classes for working with dates and times 
import datetime as dt

import matplotlib.pyplot as plt
# Plotly Express is a high-level visualization library 
import plotly.express as px
# Plotly Graph Objects is a library for creating complex visualizations 
import plotly.graph_objects as go
# This library is used to create complex subplots 
from plotly.subplots import make_subplots

# this library is used to send HTTP requests using Python
import requests

# this line imports the warnings module which provides a way to handle warnings in Python 
import warnings

# This line surppresses warnings that may be displayed during the execution of the program.  
warnings.filterwarnings('ignore')

In [24]:
START_DATE = '2022-09-15'

You can find those and other cryptocurrency datasets here: https://www.cryptodatadownload.com/data/bitfinex/. 

In [3]:
## URLS and names
urls = ["https://www.cryptodatadownload.com/cdd/Bitfinex_EOSUSD_d.csv", 
        "https://www.cryptodatadownload.com/cdd/Bitfinex_EDOUSD_d.csv",
        "https://www.cryptodatadownload.com/cdd/Bitfinex_BTCUSD_d.csv",
        "https://www.cryptodatadownload.com/cdd/Bitfinex_ETHUSD_d.csv",
        "https://www.cryptodatadownload.com/cdd/Bitfinex_LTCUSD_d.csv",
        "https://www.cryptodatadownload.com/cdd/Bitfinex_BATUSD_d.csv",
        "https://www.cryptodatadownload.com/cdd/Bitfinex_OMGUSD_d.csv",
        "https://www.cryptodatadownload.com/cdd/Bitfinex_DAIUSD_d.csv",
        "https://www.cryptodatadownload.com/cdd/Bitfinex_ETCUSD_d.csv",
        "https://www.cryptodatadownload.com/cdd/Bitfinex_ETPUSD_d.csv",
        "https://www.cryptodatadownload.com/cdd/Bitfinex_NEOUSD_d.csv",
        "https://www.cryptodatadownload.com/cdd/Bitfinex_REPUSD_d.csv",
        "https://www.cryptodatadownload.com/cdd/Bitfinex_TRXUSD_d.csv",
        "https://www.cryptodatadownload.com/cdd/Bitfinex_XLMUSD_d.csv",
        "https://www.cryptodatadownload.com/cdd/Bitfinex_XMRUSD_d.csv",
        "https://www.cryptodatadownload.com/cdd/Bitfinex_XVGUSD_d.csv", 
       ]
crypto_names = ["EOS Coin (EOS)",
                "Eidoo (EDO)",
                "Bitcoin (BTC)",
                "Ethereum (ETH)",
                "Litecoin (LTC)",
                "Basic Attention Token (BAT)",
                "OmiseGO (OMG)",
                "Dai (DAI)",
                "Ethereum Classic (ETC)",
                "Metaverse (ETP)",
                "Neo (NEO)",
                "Augur (REP)",
                "TRON (TRX)",
                "Stellar (XLM)",
                "Monero (XMR)",
                "Verge (XVG)"
               ]

### Download data

- The parameter `verify` is set to `False`, indicating that SSL certificate verification is disabled. This is useful when the SSL certificate of the website is self-signed or invalid, and we still want to make a request to the website. However, it is not recommended to disable SSL certificate verification in production environments, as it can leave the application vulnerable to man-in-the-middle attacks.

In [25]:
def df_loader(urls, start_date=START_DATE): 
    filenames=[]
    all_df = pd.DataFrame()
    
    for idx, url in enumerate(urls): 
        req = requests.get(url, verify=False)
        url_content = req.content 
        filename = url[48:]
        
        csv_file = open(filename, 'wb') 
        csv_file.write(url_content)
        csv_file.close()
        
        filename = filename[:-9]
        filenames.append(filename)
     
    for file in filenames: 
        df = pd.read_csv(file + 'USD_d.csv', header=1, parse_dates=['date'])
        df = df[df['date'] > start_date]
        df.index = df.date
        df.drop(labels=[df.columns[0], df.columns[1], df.columns[8]], axis=1, inplace=True)
        all_df = pd.concat([all_df, df], ignore_index=False)
        
    return all_df, filenames

In [26]:
all_df, filenames = df_loader(urls, start_date=START_DATE)

In [27]:
all_df

,symbol,open,high,low,close,Volume USD
date,,,,,,
2023-06-02,EOS/USD,0.884090,0.884090,0.876050,0.876050,1821.219572
2023-06-01,EOS/USD,0.891290,0.901820,0.875000,0.882990,83274.435569
2023-05-31,EOS/USD,0.912640,0.913120,0.875200,0.892420,113863.249780
2023-05-30,EOS/USD,0.897590,0.926990,0.892140,0.912800,101931.336427
2023-05-29,EOS/USD,0.910640,0.915630,0.892800,0.897690,100059.422028
...,...,...,...,...,...,...
2022-09-20,XVG/USD,0.003265,0.003523,0.003196,0.003236,5442.505788
2022-09-19,XVG/USD,0.003219,0.003320,0.003170,0.003269,11482.359139
2022-09-18,XVG/USD,0.003378,0.003705,0.003202,0.003209,9860.704663


We'll need each currency to have individual dataset. For that, we divide the original dataset into a list of datasets.  

In [57]:
crypto_df = []
for file in filenames: 
    symbol = file + '/USD'
    temp_df = all_df[all_df.symbol == symbol]
    temp_df.drop(columns=['symbol'], inplace=True)
    exec('%s = temp_df.copy()' % file.lower())
    crypto_df.append(temp_df)

In [58]:
crypto_df

[                open      high       low    close    Volume USD
 date                                                           
 2023-06-02  0.884090  0.884090  0.876050  0.87605  1.821220e+03
 2023-06-01  0.891290  0.901820  0.875000  0.88299  8.327444e+04
 2023-05-31  0.912640  0.913120  0.875200  0.89242  1.138632e+05
 2023-05-30  0.897590  0.926990  0.892140  0.91280  1.019313e+05
 2023-05-29  0.910640  0.915630  0.892800  0.89769  1.000594e+05
 ...              ...       ...       ...      ...           ...
 2022-09-20  1.299345  1.383700  1.244749  1.33810  1.265798e+06
 2022-09-19  1.272400  1.313086  1.211800  1.29800  1.047968e+06
 2022-09-18  1.471900  1.473500  1.237800  1.27290  8.968700e+05
 2022-09-17  1.424300  1.491326  1.417369  1.47060  4.393965e+05
 2022-09-16  1.383800  1.455008  1.380500  1.42280  1.043009e+06
 
 [260 rows x 5 columns],
                open     high       low    close    Volume USD
 date                                                         
 2

### Buttons and dropdown 
We will have 8 date options: All time, Last Year, Current Year, Last 2 Month, Current Month, Last 2 weeks, Last 4 days, Today. 

In [59]:
date_buttons = [
    {'step':'all', 'label':'All Time'},
    {'count': 1, 'step':'year', 'stepmode':'backward', 'label':'Last Year'},
    {'count': 1, 'step':'year', 'stepmode':'todate', 'label':'Current Year'},
    {'count': 2, 'step':'month', 'stepmode':'backward', 'label':'Last 2 Months'},
    {'count': 1, 'step':'month', 'stepmode':'todate', 'label':'Current Month'},
    {'count': 14, 'step':'day', 'stepmode':'backward', 'label':'Last 2 Weeks'},
    {'count': 4, 'step':'day', 'stepmode':'backward', 'label':'Last 4 days'},
    {'count': 1, 'step':'day', 'stepmode':'todate', 'label':'Today'}
]

In [60]:
buttons = []
i = 0
j = 0
COUNT=2
vis = [False] * len(crypto_names) * COUNT

for df in crypto_df: 
    for k in range(COUNT): 
        vis[j+k] = True 
    buttons.append({
        'label': crypto_names[i],
        'method': 'update',
        'args': [
            {'visible': vis,
             'title': crypto_names[i] + 'Charts and Indicators'
            }
        ]
    })
    i+=1
    j+=COUNT
    vis = [False] * len(crypto_names) * COUNT

### Charts

In [61]:
fig = make_subplots(rows=3,
                    cols=1,
                    shared_xaxes=True,
                    specs=[[{'rowspan':2}], [{'rowspan':1}], [{}]])

In [62]:
for df in crypto_df: 
    fig.add_trace(go.Candlestick(x=df.index, 
                                 open=df.open, 
                                 high=df.high, 
                                 low=df.low, 
                                 close=df.close,
                                 showlegend=False
                                ), 
                  row=1, 
                  col=1
    )
    fig.add_trace(go.Bar(x=df.index,
                         y=df['Volume USD'],
                         showlegend=False,
                         marker_color='aqua'
                        ),
                  row=3,
                  col=1
    )

Customise the charts and menu.

In [63]:
fig.update_layout(width=970,
                  height=650,
                  template='plotly_dark',
                  font_family='monospace',
                  xaxis=dict(rangeselector=dict(buttons=date_buttons)),
                  updatemenus=[dict(type='dropdown', 
                                    active=0, 
                                    showactive=True, 
                                    buttons=buttons,
                                    x=1.05,
                                    y=1.2)],
                  title=dict(text='<b>Cryptocurrencies Dashboard<b>',
                             font=dict(color='#FFFFFF', size=22),
                             x = 0.50),
                  font=dict(color='blue'),
                  annotations=[
                      dict(
                          text='<b>Candlestick Chart <b>',
                          font=dict(size=22, color='#FFFFFF'),
                          x=0.5,
                          y=0.285,
                          xref='paper', yref='paper',
                          showarrow=False,
                          align='left',

                      ),
                      dict(
                          text='<b>Volume Traded<b>',
                          font=dict(size=22, color='#ffffff'),
                          x=0.49,
                          y=-0.48,
                          showarrow=False,
                          xref='paper', yref='paper',
                          align='left'
                      )
                  ]
)

fig.update_layout(
    spikedistance=100,
    hoverdistance=1000,
    xaxis_rangeslider_visible=False
)

fig.update_xaxes(
    tickfont=dict(size=15, family='monospace', color='#B8B8B8'),
    tickmode='array',
    ticklen=6,
    showline=False,
    showgrid=True,
    gridcolor='#595959',
    ticks='outside',
    
    showspikes=True,
    spikesnap='cursor',
    spikemode='across'
)

fig.update_yaxes(
    tickfont=dict(size=15, family='monospace', color='#B8B8B8'), 
    tickmode='array',
    showline=False,
    ticksuffix='$',
    showgrid=True,
    gridcolor='#595959',
    ticks='outside',
    
    showspikes=True,
    spikesnap='cursor',
    spikemode='across'
)


for i in range(0,16*COUNT):
    fig.data[i].visible = False 
    
for i in range(COUNT):
    fig.data[i].visible = True
    
fig.layout["xaxis"]["rangeslider"]["visible"] = False
fig.layout["xaxis2"]["rangeslider"]["visible"] = False
fig.layout["xaxis3"]["rangeslider"]["visible"] = True

fig.show()